In [20]:
import os 
import boto3
import PyPDF2
import json 


In [ ]:
path = r"G:\My Drive\personal\My AWS Journey\Bedrock_course_by_Jose_Portilla\Amazon_Bedrock_UNZIP_ME\01-Text-Generation"
file_path = os.path.join(path, 'Corporate_Travel_Policy.pdf')

def extract_clean_text(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = reader.pages[0].extract_text()

    if text:
        lines = text.split('\n')
        cleaned = ' '.join(lines)
        return cleaned
    return ''

text = extract_clean_text(file_path)
print(text)


Corporate Travel and Time Off Policy Introduction This policy establishes clear guidelines and procedures for time off and corporate travel for employees. It aims to ensure fair and consistent application throughout the organization while supporting operational needs. Annual Paid Time Off (PTO) Entitlement ● PTO Allocation: All employees receive five weeks (25 working days) of PTO per calendar year . ● Accrual of PTO: PTO accrues monthly based on the annual entitlement. ● Carryover: Unused PTO cannot be carried over to the next year . Employees are encouraged to utilize their PTO within the accrual year . Time Off Beyond PTO ● Managerial Approval: Additional time off beyond the allocated five weeks requires prior approval from the employee's direct manager . ● Request Procedure: Submit time off requests at least four weeks in advance for any period exceeding annual PTO. ● Considerations for Approval: Managers will assess the operational impact, employee performance and attendance, and 

In [14]:
type(text)

str

In [24]:
bedrock_runtime = boto3.client(region_name='us-west-2',
                               service_name='bedrock-runtime')

In [43]:
prompt = f"Based on the information provided in the policy document: \n{text}, \nHow many working days of PTO do employees get?"

In [44]:
print(prompt)

Based on the information provided in the policy document: 
Corporate Travel and Time Off Policy Introduction This policy establishes clear guidelines and procedures for time off and corporate travel for employees. It aims to ensure fair and consistent application throughout the organization while supporting operational needs. Annual Paid Time Off (PTO) Entitlement ● PTO Allocation: All employees receive five weeks (25 working days) of PTO per calendar year . ● Accrual of PTO: PTO accrues monthly based on the annual entitlement. ● Carryover: Unused PTO cannot be carried over to the next year . Employees are encouraged to utilize their PTO within the accrual year . Time Off Beyond PTO ● Managerial Approval: Additional time off beyond the allocated five weeks requires prior approval from the employee's direct manager . ● Request Procedure: Submit time off requests at least four weeks in advance for any period exceeding annual PTO. ● Considerations for Approval: Managers will assess the op

In [25]:
body = json.dumps({"inputText": prompt,
                   'textGenerationConfig': {
                       'temperature': 0,
                        'maxTokenCount': 512
                   }
                   })

In [26]:
response = bedrock_runtime.invoke_model(body=body,
                                        modelId='amazon.titan-text-express-v1')

In [27]:
response_body = json.loads(response.get('body').read())

In [28]:
print(response_body)

{'inputTextTokenCount': 350, 'results': [{'tokenCount': 10, 'outputText': '\nFive weeks or 25 working days', 'completionReason': 'FINISH'}]}


In [29]:
print(response_body['results'][0]['outputText'])


Five weeks or 25 working days


## Put everything together in a single function 

In [39]:
def llm_helper (file_path, query, model_id):
    text = extract_clean_text(file_path)

    bedrock_runtime = boto3.client(region_name='us-west-2',
                               service_name='bedrock-runtime')
    
    
    prompt = f"Based on the information provided in the policy document: {text}, {query}"

    body = json.dumps({"inputText": prompt,
                   'textGenerationConfig': {
                       'temperature': 0,
                        'maxTokenCount': 512
                   }
                   })
    
    response = bedrock_runtime.invoke_model(body=body,
                                        modelId=model_id)
    
    response_body = json.loads(response.get('body').read())

    return response_body['results'][0]['outputText']

In [40]:
output = llm_helper(file_path, 'How many working days of PTO do employees get?', 'amazon.titan-text-express-v1')
print(output)



Five weeks or 25 working days
